In [160]:
from bs4 import BeautifulSoup as bs
import requests
from pprint import pprint
import pandas as pd
import re
from pymongo import MongoClient
import hashlib

In [161]:
vacancies = []
more_page = True

In [162]:
#Функция обработки отдельного блока вакансии
def get_vacancy_info(vacancy):
    #Название вакансии
    vacancy_name = vacancy.find('span',{'class':'resume-search-item__name'}).text
    #Зарплата
    vacancy_salary = vacancy.find('div',{'class':'vacancy-serp-item__sidebar'}).text.replace("\xa0","")
    if not vacancy_salary or vacancy_salary == 'По договоренности':
        salary_min = None
        salary_max = None
        salary_cur = None
    elif re.findall(r'^\w+', vacancy_salary)[0] == 'от':
        salary_min = int(re.findall(r'\d+', vacancy_salary)[0])
        salary_max = None
        salary_cur = re.findall(r'\w+', vacancy_salary)[-1]
    elif re.findall(r'^\w+', vacancy_salary)[0] == 'до':
        salary_max = int(re.findall(r'\d+', vacancy_salary)[0])
        salary_min = None
        salary_cur = re.findall(r'\w+', vacancy_salary)[-1]
    else:
        salary_min = int(re.findall(r'\w+', vacancy_salary)[0])
        salary_max = int(re.findall(r'\w+', vacancy_salary)[1])
        salary_cur = re.findall(r'\w+', vacancy_salary)[-1]
    
    #Ссылка
    vacancy_url = vacancy.find('span',{'class':'resume-search-item__name'}).a['href']
    #хеш документа
    vac_hash = hashlib.md5()
    vac_hash.update(repr({'name' : vacancy_name, 'salary_min' : salary_min, 'salary_max' : salary_max, 'salary_cur' : salary_cur, 'url' :vacancy_url}).encode('utf-8'))
    

    return {'name' : vacancy_name, 'salary_min' : salary_min, 'salary_max' : salary_max, 'salary_cur' : salary_cur, 'url' :vacancy_url, 'hash': vac_hash.hexdigest()}

In [163]:
#Функция обработки отдельных страниц
def parse_page(url):
    print(f'Parsing url: {url}')
    
    response = requests.get(url,headers=headers)
    soup = bs(response.text, 'lxml')      
    vacancy_all = soup.findAll('div', {'class':'vacancy-serp-item'})
    for vac in vacancy_all:
      vacancies.append(get_vacancy_info(vac))
    try:
        # проверяем есть ли кнопка Далее, и если есть возращаем линк, если нет возращаем False
        new_link = 'https://hh.ru' + soup.find('a',{'class':'HH-Pager-Controls-Next'})['href']
        return new_link
    except:
        return False

In [164]:
job_title = input('Введите название должности: ')
main_link = 'https://hh.ru/search/vacancy'
params = {'text':job_title, 'page' :'0'}
headers = {'User-Agent': 'Mozilla/5.0'}

In [166]:
response = requests.get(main_link,params=params, headers=headers)
soup = bs(response.text, 'lxml')
vacancy_all = soup.findAll('div', {'class':'vacancy-serp-item'})
for vac in vacancy_all:
    vacancies.append(get_vacancy_info(vac))

In [167]:
# Проверяем есть ли клавиша Далее, и даем либо новый линк, либо завершаем
try:
    new_link = 'https://hh.ru' + soup.find('a',{'class':'HH-Pager-Controls-Next'})['href']
except:
    more_page = False
while more_page:
    if not new_link:
        break
    new_link = parse_page(new_link)
     

Parsing url: https://hh.ru/search/vacancy?L_is_autosearch=false&clusters=true&enable_snippets=true&text=Data+Science&page=1
Parsing url: https://hh.ru/search/vacancy?L_is_autosearch=false&clusters=true&enable_snippets=true&text=Data+Science&page=2
Parsing url: https://hh.ru/search/vacancy?L_is_autosearch=false&clusters=true&enable_snippets=true&text=Data+Science&page=3
Parsing url: https://hh.ru/search/vacancy?L_is_autosearch=false&clusters=true&enable_snippets=true&text=Data+Science&page=4
Parsing url: https://hh.ru/search/vacancy?L_is_autosearch=false&clusters=true&enable_snippets=true&text=Data+Science&page=5
Parsing url: https://hh.ru/search/vacancy?L_is_autosearch=false&clusters=true&enable_snippets=true&text=Data+Science&page=6
Parsing url: https://hh.ru/search/vacancy?L_is_autosearch=false&clusters=true&enable_snippets=true&text=Data+Science&page=7
Parsing url: https://hh.ru/search/vacancy?L_is_autosearch=false&clusters=true&enable_snippets=true&text=Data+Science&page=8
Parsing 

In [168]:
print(f'Всего найдено {len(vacancies)} вакансий по запросу {job_title}')

Всего найдено 733 вакансий по запросу Data Science


In [169]:
vacancies[1]

{'name': 'ML Разработчик / Data scientist',
 'salary_min': None,
 'salary_max': None,
 'salary_cur': None,
 'url': 'https://hh.ru/vacancy/37422856?query=Data%20Science',
 'hash': '5c901c8520569452c62cba9199efc506'}

In [170]:
def store_vacancies_mongo(host, port, vacs):    
    # Функция принимает 3 аргумента: host - адрес сервера Mongodb, port - порт, vacs - список словарей вакансий
    # Проверка дубликатов идет через хеш документа
    client = MongoClient(host, port)
    db = client['GB']
    vacancies_grab = db.vacancies
    for vac in vacancies:
        vacancies_grab.update_one({'hash': vac['hash']},{'$set': vac}, upsert=True)
    print(f'Всего в базе {vacancies_grab.count_documents({})} документов')

In [133]:
vacancies_grab.find({'$or': [{'salary_max': {'$gt': 100000}},{'salary_min':{'$gt': 100000}}]}, {'name': 1, 'salary_min': 1,'salary_max': 1,'salary_cur': 1,'url': 1, '_id': 0}).count()

<ipython-input-133-0d69fb3f4946>:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  vacancies_grab.find({'$or': [{'salary_max': {'$gt': 100000}},{'salary_min':{'$gt': 100000}}]}, {'name': 1, 'salary_min': 1,'salary_max': 1,'salary_cur': 1,'url': 1, '_id': 0}).count()


61

In [171]:
def search_salary_gt(salary):
    finded = vacancies_grab.find({'$or': [{'salary_max': {'$gt': salary}},{'salary_min':{'$gt': salary}}]}, {'name': 1, 'salary_min': 1,'salary_max': 1,'salary_cur': 1,'url': 1, '_id': 0})
    num = vacancies_grab.count_documents({'$or': [{'salary_max': {'$gt': salary}},{'salary_min':{'$gt': salary}}]})
    print(f'Всего найдена {num} с зарплатой больше {salary}')
    for vac in finded:
        print(vac)

In [172]:
store_vacancies_mongo('127.0.0.1',27017, vacancies)

Всего в базе 730 документов


In [173]:
search_salary_gt(300000)

Всего найдена 3 с зарплатой больше 300000
{'name': 'Разработчик', 'salary_cur': 'KZT', 'salary_max': 1000000, 'salary_min': 500000, 'url': 'https://hh.ru/vacancy/37111876?query=Data%20Science'}
{'name': 'Public Policy Research Analyst (Russian speaking, freelance)', 'salary_cur': 'руб', 'salary_max': 800000, 'salary_min': 200000, 'url': 'https://hh.ru/vacancy/37373568?query=Data%20Science'}
{'name': 'Fullstack JavaScript разработчик', 'salary_cur': 'KZT', 'salary_max': 700000, 'salary_min': 300000, 'url': 'https://hh.ru/vacancy/37112096?query=Data%20Science'}
